<a href="https://colab.research.google.com/github/May-Xiaoting-Zhou/-CS577_Deep-Learning_Project/blob/main/code/matrix_and_tracking_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Importing the necessary libraries
import pandas as pd
import datetime
import json
import os
import csv
import time
import re

output_folder = '/content/drive/MyDrive/MX_Project_Data'

In [2]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [4]:
def extract_values(string):
  pattern = r':\s*(\d+),*'
  matches = re.findall(pattern, string)
  return [int(match) for match in matches]

In [23]:
for st in range(1, 2):
  for ss in range(1, 4):
    if st == 1 and ss == 1:
      continue
    folder_path = output_folder + '/Session' + str(ss) + '_Raw'
    file_name = 'df_sub' + str(st) + '_ss' + str(ss) + '_tracking.csv'
    file_path = os.path.join(folder_path, file_name)

    df = pd.read_csv(file_path, index_col=0)

    df[['headPosition_x', 'headPosition_y', 'headPosition_z']] = df['headPosition'].apply(lambda x: pd.Series(str(x).strip('][').split(',')))
    df['headPosition_x'] = df['headPosition_x'].apply(lambda x: x[5:])
    df['headPosition_y'] = df['headPosition_y'].apply(lambda x: x[5:])
    df['headPosition_z'] = df['headPosition_z'].apply(lambda x: x[5:-1])

    df['matrix'] = df['matrix'].apply(lambda x: extract_values(x))
    df['time_unix'] = df['time'].apply(lambda x: int(datetime.datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p').timestamp()))
    df.drop(['headPosition'], axis=1, inplace=True)

    df['subject'] = st
    df['session'] = ss

    df.to_csv(file_path, index=False)

In [3]:

df = pd.read_csv('/content/drive/MyDrive/MX_Project_Data/Session1_Raw/df_sub15_ss1_tracking_puzzle_datetimes.csv', index_col=0)
df

,puzzle,datatime_from,datatime_to,datatime_stamp_from,datatime_stamp_to,time_difference
1,1,9/21/2022 4:28:11 PM,9/21/2022 4:28:57 PM,1663777691,1663777737,46
2,2,9/21/2022 4:29:17 PM,9/21/2022 4:29:47 PM,1663777757,1663777787,30
3,3,9/21/2022 4:30:07 PM,9/21/2022 4:33:09 PM,1663777807,1663777989,182


# player_puzzle_assessment

In [ ]:
for ss in range(1, 4):
  for st in range(1, 74):

    df_player_assessment = pd.read_csv('/content/drive/MyDrive/MX_Project_Data/Session' + str(ss) + '/df_sub' + str(st) + '_ss' + str(ss) + '_player_assessment_csv.csv')
    df_puzzle = pd.read_csv('/content/drive/MyDrive/MX_Project_Data/Session' + str(ss) + '/df_sub' + str(st) + '_ss' + str(ss) + '_puzzle_datetimes.csv')

    df_puzzle = df_puzzle.rename(columns={'puzzle': 'number_of_completed_puzzle'})
    df_puzzle['time_skip'] = 0
    for rownum in range(0, len(df_puzzle) - 1):
        df_puzzle.loc[rownum, 'time_skip'] = df_puzzle.iloc[rownum+1]['datatime_stamp_from'] - df_puzzle.iloc[rownum]['datatime_stamp_to']

    df_player_puzzle_assessment = df_player_assessment.merge(df_puzzle, on=['subject', 'session', 'number_of_completed_puzzle'], how='left')
    df_player_puzzle_assessment.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1, inplace=True)

    df_player_puzzle_assessment.to_csv('/content/drive/MyDrive/MX_Project_Data/Session' + str(ss) + '/df_sub' + str(st) + '_ss' + str(ss) + '_puzzle_player_assessment.csv', index=False)


In [ ]:
st_, ss_

(73, 3)

In [ ]:
# last: to csv
df_player_assessment.to_csv('/content/drive/MyDrive/MX_Project_Data/Session' + str(ss_) + '/df_sub' + str(st_) + '_ss' + str(ss_) + '_player_assessment_csv.csv', index=False)


In [ ]:
idx = 4
pzn = df_player_assessment.loc[idx, "puzzle_ID"]
df_player_assessment.loc[idx, "score"] = 720

df_player_assessment

,Unnamed: 0,subject,session,answerq1,answerq2,puzzle_ID,number_of_completed_puzzle,score,time
0,346,67,3,1,1,72,1,401,1668012633
1,347,67,3,1,1,53,2,483,1668012763
2,348,67,3,1,1,91,3,559,1668012928
3,349,67,3,1,1,44,4,636,1668013086
4,350,67,3,1,1,1,5,720,1668013228


In [ ]:
idx = 4
pzn = df_player_assessment.loc[idx, "puzzle_ID"]
df_player_assessment.loc[idx, "number_of_completed_puzzle"] = idx + 1
df_player_assessment.loc[idx, "score"] = avg_score(pzn, idx+1)
df_player_assessment.loc[idx, "time"] = df_puzzle_datetimes.loc[idx, "datatime_stamp_to"] + 43200.0

df_player_assessment

,Unnamed: 0,subject,session,answerq1,answerq2,puzzle_ID,number_of_completed_puzzle,score,time
0,346,67,3,1,1,72,1,401,1668012633
1,347,67,3,1,1,53,2,483,1668012763
2,348,67,3,1,1,91,3,559,1668012928
3,349,67,3,1,1,44,4,636,1668013086
4,350,67,3,1,1,1,5,NaN,1668013228


In [ ]:
df_player_assessment

,Unnamed: 0,subject,session,answerq1,answerq2,puzzle_ID,number_of_completed_puzzle,score,time
0,346,67,3,1,1,72,1,401,1668012633
1,347,67,3,1,1,53,2,483,1668012763
2,348,67,3,1,1,91,3,559,1668012928
3,349,67,3,1,1,44,4,636,1668013086
4,350,67,3,1,1,1,5,NaN,1668002428


In [ ]:
df_puzzle_datetimes

,Unnamed: 0,subject,session,puzzle,datatime_from,datatime_to,datatime_stamp_from,datatime_stamp_to,time_difference
0,1,67,3,1,11/9/2022 4:48:37 PM,11/9/2022 4:50:33 PM,1667969317,1667969433,116
1,2,67,3,2,11/9/2022 4:50:53 PM,11/9/2022 4:52:43 PM,1667969453,1667969563,110
2,3,67,3,3,11/9/2022 4:53:03 PM,11/9/2022 4:55:28 PM,1667969583,1667969728,145
3,4,67,3,4,11/9/2022 4:55:51 PM,11/9/2022 4:58:04 PM,1667969751,1667969884,133
4,5,67,3,5,11/9/2022 4:58:31 PM,11/9/2022 5:00:28 PM,1667969911,1667970028,117


In [ ]:
for i in range(0, len(df_puzzle_datetimes)):
  print(df_player_assessment.loc[i, "time"] - df_puzzle_datetimes.loc[i, "datatime_stamp_to"])

43200.0
43200.0
43200.0
43202.0
32400.0


In [ ]:
df_player_assessment.loc[2, "number_of_completed_puzzle"] = 3
df_player_assessment.loc[2, "score"] = avg_score(7, 3)
df_player_assessment.loc[2, "time"] = df_puzzle_datetimes.loc[2, "datatime_stamp_to"]

df_player_assessment

,Unnamed: 0,subject,session,answerq1,answerq2,puzzle_ID,number_of_completed_puzzle,score,time
0,613,10,1,0,0,68,1,77,1663427243
1,614,10,1,0,0,28,2,156,1663427327
2,615,10,1,1,1,7,3,277,1663384314


In [ ]:
avg_score(7, 3)

276.62857142857143

In [ ]:
df_puzzle_datetimes= pd.read_csv('/content/drive/MyDrive/MX_Project_Data/Session1/df_sub10_ss1_puzzle_datetimes.csv')
df_puzzle_datetimes.head()

,Unnamed: 0,subject,session,puzzle,datatime_from,datatime_to,datatime_stamp_from,datatime_stamp_to,time_difference
0,1,10,1,1,9/17/2022 3:06:29 PM,9/17/2022 3:07:23 PM,1663383989,1663384043,54
1,2,10,1,2,9/17/2022 3:07:56 PM,9/17/2022 3:08:47 PM,1663384076,1663384127,51
2,3,10,1,3,9/17/2022 3:09:07 PM,9/17/2022 3:11:54 PM,1663384147,1663384314,167


# matrix

In [ ]:
# Read a Json File to a Dataframe
def readJson_PUZZLE(directory, fileName):
  df = pd.DataFrame()
  df['PUZZLE'] = 0
  with open(directory + '/' + fileName, 'r') as f:
    puzzle = 1
    for line in f:
      if 'NEW_PUZZLE' in line:
        puzzle += 1
      else:
        data = json.loads(line)
        data['PUZZLE'] = puzzle
        df = df.append(data, ignore_index=True)
  return df

In [ ]:
# prompt: read  the file name that is end with "matrix_easy.json" under "/content/drive/MyDrive/MX_Project_Data/Subject_X/Session_X/", which X is from 1 to 73
for subject in range(1, 74):
  for session in range(1, 4):
    df = pd.DataFrame()
    df_toCSV = pd.DataFrame()
    directory = f'/content/drive/MyDrive/MX_Project_Data/Subject_{subject}/Session_{session}/'
    for filename in os.listdir(directory):
      if session == 1 and filename.endswith('matrix_easy.json'):
          df = readJson_PUZZLE(directory, filename)
      elif session == 2 and filename.endswith('matrix_medium.json'):
          df = readJson_PUZZLE(directory, filename)
      elif session == 3 and filename.endswith('matrix_hard.json'):
          df = readJson_PUZZLE(directory, filename)

      print(df)
      if not df.empty:
        grouped_df = df.groupby('PUZZLE')
        max_min_time = grouped_df['time'].agg(['max', 'min'])
        group = pd.DataFrame(max_min_time)
        print(group)

        for index, row in group.iterrows():
          max_dt = datetime.datetime.strptime(row['max'], '%m/%d/%Y %H:%M:%S %p').timestamp()
          min_dt = datetime.datetime.strptime(row['min'], '%m/%d/%Y %H:%M:%S %p').timestamp()
          df_toCSV["puzzle"] = index
          df_toCSV["datatime_from"] = row['max']
          df_toCSV["datatime_to"] = row['min']
          df_toCSV["datatime_stamp_from"] = max_dt
          df_toCSV["datatime_stamp_to"] = min_dt
          df_toCSV["time_difference"] = max_dt-min_dt
        print(df_toCSV)
        break
        df_toCSV.to_csv(output_folder + '/Session' + str(session) + '/df_sub' + str(subject) + '_ss' + str(session) + '_puzzle_datetimes.csv')

    break



In [ ]:
df_toCSV

,puzzle,datatime_from,datatime_to,datatime_stamp_from,datatime_stamp_to,time_difference


In [ ]:
# generate matrix data features by grouping timestamp
# (1, 1)
for ss in range(1, 4):
  for st in range(1, 74):
    df_matrix = pd.read_csv(output_folder + '/Session' + str(ss) + '/df_sub' + str(st) + '_ss' + str(ss) + '_matrix_csv.csv')

    df_matrix = df_matrix.drop(['Unnamed: 0'], axis=1)
    # Group the dataframe by 'subject', 'session', and 'timestamp'
    grouped = df_matrix.groupby(['subject', 'session', 'matrixWithNumber'])

    # Calculate the mean, median, max, and min of confidence
    df_matrix_agg = grouped['time'].agg(['max', 'min'])
    df_matrix_agg['time_span'] = grouped['time'].apply(lambda x:  max(x) - min(x))

    # Reset the index to make 'subject', 'session', and 'timestamp' columns
    df_matrix_agg = df_matrix_agg.reset_index()

    df_matrix_agg.rename(columns={ 'max': 'max_time', 'min': 'min_time'}, inplace=True)

    # convert data frame to csv file
    df_matrix_agg.to_csv(output_folder + '/Session' + str(ss) + '/df_sub' + str(st) + '_ss' + str(ss) + '_matrix_features.csv')

    try:
      os.remove(output_folder + '/Session' + str(ss) + '/df_sub' + str(st) + '_ss' + str(ss) + '_matrix_csv.csv')
    except FileNotFoundError:
      print('Error: The specified file does not exist.', st, ss)


# tracking

In [ ]:
def df_group(grouped, col):
    df_agg = grouped[col].agg(['mean', 'median', 'max', 'min'])
    df_agg.rename(columns={'mean': 'mean_' + col, 'median': 'median_' + col, 'max': 'max_' + col, 'min': 'min_' + col}, inplace=True)
    return df_agg

In [ ]:
# generate tracking data features by grouping timestamp
# (1, 1)
for ss in range(1, 4):
  for st in range(1, 74):
    df_tracking = pd.read_csv(output_folder + '/Session' + str(ss) + '/df_sub' + str(st) + '_ss' + str(ss) + '_tracking_csv.csv')

    df_tracking = df_tracking.drop(['Unnamed: 0'], axis=1)
    # Group the dataframe by 'subject', 'session', and 'timestamp'
    grouped = df_tracking.groupby(['subject', 'session', 'matrix'])

    # Calculate the mean, median, max, and min of each column
    # time
    df_tracking_agg_0 = grouped['time'].agg(['max', 'min'])
    df_tracking_agg_0['time_span'] = grouped['time'].apply(lambda x:  max(x) - min(x))
    df_tracking_agg_0.rename(columns={ 'max': 'max_time', 'min': 'min_time'}, inplace=True)

    # headPosition_x
    df_tracking_agg_1 = df_group(grouped, 'headPosition_x')
    # headPosition_y
    df_tracking_agg_2 = df_group(grouped, 'headPosition_y')
    # headPosition_z
    df_tracking_agg_3 = df_group(grouped, 'headPosition_z')
    # headPosition_magnitude
    df_tracking_agg_4 = df_group(grouped, 'headPosition_magnitude')

    df_tracking_agg = pd.concat([df_tracking_agg_0, df_tracking_agg_1, df_tracking_agg_2, df_tracking_agg_3,df_tracking_agg_4], axis=1, join='inner')
    # Reset the index to make 'subject', 'session', and 'timestamp' columns
    df_tracking_agg = df_tracking_agg.reset_index()

    # convert data frame to csv file
    df_tracking_agg.to_csv(output_folder + '/Session' + str(ss) + '/df_sub' + str(st) + '_ss' + str(ss) + '_tracking_features.csv')

    try:
      os.remove(output_folder + '/Session' + str(ss) + '/df_sub' + str(st) + '_ss' + str(ss) + '_tracking_csv.csv')
    except FileNotFoundError:
      print('Error: The specified file does not exist.', st, ss)
